# Lab 03: Cây quyết định

- Họ tên: Nguyễn Tường Bách Hỷ
- MSSV: 22120455

<font color='red'>**LƯU Ý:**</font>

- Bài làm của sinh viên phải do chính bản thân sinh viên tự làm, có thể trao đổi và tham khảo ý tưởng nhưng không được sao chép (một phần hoặc toàn bộ) code hoặc lời giải từ bất cứ người nào khác. Nếu vi phạm sẽ bị <font color='red'>0đ</font> bài tập này.
- Các bạn có thể tạo thêm các cell trong quá trình code, tuy nhiên các bạn vui lòng <font color='red'>không xóa các cell code mặc định và các cell test case</font> (vì có thể ảnh hưởng đến kết quả khi chấm bài).
- Các test case được đưa ra chỉ nhằm mục đích giúp các bạn test code của mình, <font color='red'>**việc pass các test case này không đồng nghĩa với việc lời giải của các bạn sẽ đạt điểm tối đa**</font>. 

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; trong file, từ `TODO` để cho biết những phần mà bạn cần phải làm.

**Cách nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart Kernel & Run All Cells`, để restart và chạy tất cả các cell trong notebook của các bạn; do đó, trước khi nộp bài, các bạn nên chạy thử `Kernel` - `Restart Kernel & Run All Cells` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, các bạn tạo thư mục nộp bài theo cấu trúc sau:

- Thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`)
    - File `HW03.ipynb` (Đổi tên file notebook này theo MSSV của bạn)

Cuối cùng, các bạn nén thư mục `MSSV` này lại và nộp ở link trên moodle. **Đuôi của file nén phải là .zip (chứ không được .rar hay gì khác).**

<font color=red>Các bạn lưu ý tuân thủ chính xác qui định nộp bài ở trên.</font>

**Nội dung bài tập**

Bài tập 3 là bài tập cá nhân. Trong bài này, bạn sẽ cài đặt thuật toán học máy: 
1. Cây quyết định (Decision tree)

### Tải những thư viện cần thiết

In [121]:
import os
from zlib import adler32
from typing import Tuple, List
import requests
import shutil
import random

import scipy
import numpy as np
import sklearn.datasets as datasets
import matplotlib.pyplot as plt

In [122]:
# Init seed
seed = 2024
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

### Tải tập dữ liệu về thuốc

Hãy tưởng tượng rằng bạn là một nhà nghiên cứu y khoa đang biên soạn dữ liệu cho một nghiên cứu. Bạn đã thu thập dữ liệu về một nhóm bệnh nhân, tất cả đều mắc cùng một căn bệnh. Trong quá trình điều trị, mỗi bệnh nhân đều đáp ứng với một trong 5 loại thuốc, Thuốc A, Thuốc B, Thuốc c, Thuốc x và thuốc y.

Một phần công việc của bạn là xây dựng một mô hình để tìm ra loại thuốc nào có thể phù hợp với bệnh nhân tương lai mắc cùng một căn bệnh. Các đặc điểm của tập dữ liệu này là Tuổi, Giới tính, Huyết áp và Cholesterol của bệnh nhân, và mục tiêu là loại thuốc mà mỗi bệnh nhân đáp ứng.

Đây là một mẫu của bộ phân loại đa lớp và bạn có thể sử dụng tập dữ liệu để xây dựng một cây quyết định, sau đó sử dụng nó để dự đoán lớp của một bệnh nhân chưa biết hoặc kê đơn thuốc cho một bệnh nhân mới.

### Đọc tập dữ liệu về thuốc

Viết hàm `drug_dataloader` để load tập dữ liệu từ tập tin `drug200_numeric.csv` và trả về hai mảng numpy array `features` và `classes` đại diện cho đặc trưng và lớp tương ứng. Lưu ý, không sử Pandas. 

In [123]:
def drug_dataloader(file_path="drug200_numeric.csv"):
    features, classes = [], []

    with open(file_path, 'r') as f:
        lines = f.readlines()

    lines = lines[1:]

    for line in lines:
        values = line.split(',')
        age = int(values[0])
        sex = int(values[1])
        bp = int(values[2])
        cholesterol = int(values[3])
        na_to_k = float(values[4])

        class_value = int(values[5])

        features.append(np.array([age, sex, bp, cholesterol, na_to_k]))
        classes.append(class_value)

    return np.array(features), np.array(classes)

features, classes = drug_dataloader()

# print(np.matrix(features))
# print(np.matrix(classes))


In [124]:
# TEST
assert (
    adler32(str(features[0][:5] + features[1][:5] + features[2][:5]).encode())
    == 2319320757
)

### Phân chia tập train-valid-test

In [125]:
def train_test_split(
    X: np.ndarray, y: np.ndarray, test_ratio=0.33
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Function for proceduring train, and test sets.

    Args:
        X (np.ndarray): input features.
        y (np.ndarray): input labels.
        test_ratio (float, optional): ratio size for test sets. Defaults to 0.33.

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]: features, label for train, test respectively. 
        They are represented as numpy array.
    """
    indices = np.random.permutation(len(X))
    train_size = int(len(X) * (1 - test_ratio))


    X_train, X_test = X[indices[:train_size]], X[indices[train_size:]]
    y_train, y_test = y[indices[:train_size]], y[indices[train_size:]]

    return (
        X_train,
        X_test,
        y_train,
        y_test,
    )

# Phân chia tập train, test.
X_train, X_test, y_train, y_test = train_test_split(features, classes)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((134, 5), (66, 5), (134,), (66,))

In [126]:
# TEST
SP = X_train.shape + X_test.shape + y_train.shape + y_test.shape
assert adler32(str(SP).encode()) == 889979968

## Cây quyết định

### Độ lợi thông tin

Thông tin kỳ vọng (entropy):

$$Entropy=-\sum_{i}^{n}p_ilog_{2}(p_i)$$

Hàm entropy đạt giá trị nhỏ nhất nếu có một giá trị $p_i=1$, đạt giá trị lớn nhất nếu tất cả các $p_i$ bằng nhau. Những tính chất này của hàm entropy khiến nó được sử dụng trong việc đo độ hỗn loạn của một phép phân chia của ID3.

In [127]:
def entropy(counts: List, n_samples: int) -> float:
    """Function for calculating entropy

    Args:
        counts (list): list number of samples in each class.
        n_samples (int): number of data samples.

    Return:
        entropy (float).
    """
    if n_samples == 0:
        return 0.0

    counts = np.array(counts)
    probabilities = counts / n_samples
    probabilities = probabilities[probabilities > 0]
    # print(probabilities)
    entropy_value = np.sum(-probabilities * np.log2(probabilities))

    return entropy_value

In [128]:
def entropy_of_one_division(division: np.ndarray) -> Tuple[float, int]:
    """Function for calculating entropy of a divided group of data.

    Please note that data may have multiple classes.

    Args:
        division (np.ndarray): input divided group of data.

    Returns:
        Tuple[float, int]: entropy of a divided group of data.
    """
    n_samples = len(division)
    n_classes = set(division)

    counts = []
    
    for n_class in n_classes:
        count = np.sum(division == n_class)
        counts.append(count)

    # print(counts)
    
    return entropy(counts, n_samples), n_samples


# print(entropy_of_one_division(np.array([1, 2, 3, 2, 1, 1, 5, 4])))

In [129]:
def get_entropy(y_predict: np.ndarray, y: np.ndarray):
    """Get entropy for a split.

    Args:
        y_predict (np.ndarray): the split decision by cutoff, True/Fasle.
        y (np.ndarray): grouth truth.

    Returns:
        s (float): entropy of input split, as real-number represented with float type.
    """
    n = len(y)
    entropy_true, n_true = entropy_of_one_division(
        y[y_predict]
    )  # left hand side entropy
    
    entropy_false, n_false = entropy_of_one_division(
        y[~y_predict]
    )  # right hand side entropy
    # overall entropy
    
    # s=?
    s = (n_true / n) * entropy_true + (n_false / n) * entropy_false
    return s

Độ lợi thông tin phân lớp tập D theo thuộc tính A:
$$ Gain(A)=Entropy(D)-Entropy_{A}(D)$$

Trong ID3, tại mỗi node, thuộc tính được chọn được xác định dựa trên là thuộc tính khiến cho information gain đạt giá trị lớn nhất.

Các thuộc tính của tập Iris đều có giá trị liên tục. Do đó ta cần rời rạc hóa cho từng thuộc tính. Cách đơn giản là sử dụng một ngưỡng `cutoff` chia giá trị của dữ liệu trên mỗi thuộc tính sẽ làm 2 phần: `<cutoff` và `>=cutoff`.

Để tìm ngưỡng `cutoff` tốt nhất cho mỗi thuộc tính ta lần lượt thay `cutoff` bằng các giá trị của thuộc tính sau đó tính entropy, `cutoff` tốt nhất khi entropy bé nhất 

$$\left(\arg\min Entropy_{A}(D)\right)$$

### Cài đặt

#### Tìm một phân hoạch với một độ lợi thông tin cho trước

In [130]:
def find_best_split(
    col_data: np.ndarray, y: np.ndarray
) -> Tuple[np.float64, np.float64]:
    """Function for calculating minimum entropy for a given attributes and its label.

    Args:
        col_data (np.ndarray): input column data in training dataset.
        y (np.ndarray): given label in the training dataset.

    Returns:
        min_entropy, cutoff (Tuple[np.float64, np.float64]): the minimum entropy, and cut-off value.
    """
    min_entropy = float("inf")
    cutoff = None

    # Loop through col_data find cutoff where entropy is minimum
    for value in set(col_data):
        y_predict = col_data < value
        my_entropy = get_entropy(y_predict, y)

        if my_entropy == 0:
            return my_entropy, value
        
        if my_entropy < min_entropy:
            min_entropy = my_entropy
            cutoff = value

    # Return min entropy, and cutoff
    return min_entropy, cutoff

In [131]:
def find_best_split_of_all(
    X: np.ndarray, y: np.ndarray
) -> Tuple[np.float64, np.float64, np.float64]:
    """Function for finding one split given an information gain.

    Args:
        X (np.ndarray): input training dataset.
        y (np.ndarray): given label in the training dataset.

    Returns:
        col_idx, cutoff, min_entropy (Tuple[np.float64, np.float64, np.float64]): return the index 
        of column with minimum entropy and cut-off value.
    """
    col_idx = None
    min_entropy = float("inf")
    cutoff = None

    for idx, col_data in enumerate(X.T):
        entropy, cur_cutoff = find_best_split(col_data, y)

        if entropy == 0:  # best entropy
            return idx, cur_cutoff, entropy
        elif entropy <= min_entropy:
            min_entropy = entropy
            col_idx = idx
            cutoff = cur_cutoff

    return col_idx, cutoff, min_entropy

#### Khớp dữ liệu

In [132]:
def dtfit(X: np.ndarray, y: np.ndarray, depth: int = 0) -> dict:
    """Function for data-fitting with Decision Tree

    Node: each node represented by cutoff value and column index, value and children.
         - cutoff value is thresold where you divide your attribute,
         - column index is your data attribute index,
         - value of node is mean value of label indexes,
           if a node is leaf all data samples will have same label.

    Note that: we divide each attribute into 2 part => each node will have 2 children: left, right.

    Args:
        X (np.ndarray): training data
        y (np.ndarray): label of training data
        depth (int, optional): depth of decision tree after training. Defaults to 0.

    Returns:
        node (dict): return the node that contains cutoff value and column index, value and children.
    """
    # Stop conditions: if all value of y are the same
    if np.all(y == y[0]):
        return {"val": y[0]}

    # find one split given an information gain
    col_idx, cutoff, entropy = find_best_split_of_all(X, y)
    
    y_left = y[X[:, col_idx] < cutoff]
    y_right = y[X[:, col_idx] >= cutoff]
    X_left = X[X[:, col_idx] < cutoff]
    X_right = X[X[:, col_idx] >= cutoff]

    node = {
        "index_col": col_idx,
        "cutoff": cutoff,
        "val": np.mean(y),
        "left": None,
        "right": None
    }

    node["left"] = dtfit(X_left, y_left, depth + 1)
    node["right"] = dtfit(X_right, y_right, depth + 1)

    return node

In [133]:
# Khớp dữ liệu
tree = dtfit(X_train, y_train)

# import pprint

# pprint.pprint(tree)

#### Dự đoán

In [134]:
def _dtpredict(tree: dict, row: np.ndarray) -> int:
    """Function for making prediction use trained DT tree on given observation.

    Args:
        tree (dict): trained DT model which presented as a dict.
        row (np.ndarray): given observation which presented as numpy's array.

    Returns:
        val (int): return the value of node which is mean value of label indexes.
    """
    cur_layer = tree
    while "cutoff" in cur_layer:
        if row[cur_layer["index_col"]] < cur_layer["cutoff"]:
            cur_layer = cur_layer["left"]
        else:
            cur_layer = cur_layer["right"]

    return cur_layer.get("val", False)

In [135]:
# TEST
given_observation = features[81]
groud_truth = classes[81]

assert groud_truth == _dtpredict(tree, given_observation)

In [136]:
def dtpredict(tree: dict, data: np.ndarray) -> List:
    """Function for making prediction with trained DT model on given input observations.

    Args:
        tree (dict): trained DT model which presented as a dict.
        data (np.ndarray): input input observations.

    Returns:
        pred (List): list of predicted label for input observations.
    """
    pred = []
    for _, col in enumerate(data):
        pred.append(_dtpredict(tree, col))
    return pred

In [137]:
# TEST
given_observations = features[81:118]
groud_truth = classes[81:118]

res = (groud_truth == dtpredict(tree, given_observations))

print(res)

assert adler32(str(res).encode()) == 794314306

[ True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True]


#### Đánh giá mô hình

In [138]:
def tpfptnfn_cal(
    y_test: np.ndarray, y_pred: np.ndarray, positive_class: int = 1
) -> Tuple[int, int, int, int]:
    """Function for calculating elements of confusion matrix.

    Args:
        y_test (np.ndarray): groud truth.
        y_pred (np.ndarray): predicted label.
        positive_class (int, optional): wanted calculating class. Defaults to 1.

    Returns:
        true_positives, false_positives, true_negatives, false_negatives (Tuple[int, int, int, int]): Four 
        basic number for constructing confusion matrix  including true positives, false positives, true negatives, 
        and false negatives.
    """
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0

    for true, pred in zip(y_test, y_pred):
        if pred == positive_class:
            if true == positive_class:
                true_positives += 1
            else:
                false_positives += 1
        
        else:
            if true == positive_class:
                false_negatives += 1

            else:
                true_negatives += 1


    return true_positives, false_positives, true_negatives, false_negatives

In [139]:
# TEST
tp, fp, tn, fn = tpfptnfn_cal([1, 0, 1, 1, 0, 0, 1], [1, 0, 0, 1, 0, 1, 1])
assert adler32((str(tp) + str(fp) + str(tn) + str(fn)).encode()) == 33030344

In [140]:
# Dự đoán
pred = dtpredict(tree, X_test)

In [141]:
def calculate_metrics(
    y_test: np.ndarray, y_pred: np.ndarray
) -> Tuple[float, float, float, float]:
    """Function for calculating metrics of classification problem including accuracy, recall, 
    precision, and f1-score.

    Args:
        y_test (np.ndarray): groud truth.
        y_pred (np.ndarray): predicted label.

    Returns:
        acc, precision, recall, f1 (Tuple[float, float, float, float]): return four values 
        for each metric: accuracy, recall, precision, and f1-score.
    """
    acc = 0
    precision = 0
    recall = 0
    f1 = 0

    noc = len(np.unique(y_test))  # number of classes

    tp, fp, tn, fn = tpfptnfn_cal(y_test, y_pred)

    acc = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) > 0 else 0

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0

    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 9

    return acc, precision, recall, f1

In [142]:
acc, prec, rec, f1 = calculate_metrics(y_test, pred)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1-score:", f1)


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0


### Nghiên cứu về ý nghĩa các độ đo

Bạn hãy trình ý nghĩa từng độ đo, bao gồm accuracy, precision, recall và f-score.

<h1>Ý nghĩa của từng độ đo</h1>

<h3 style="font-weight: 900">I. Accuracy (Độ chính xác tổng thể):</h3>
<h5 style="font-weight: 900">1.Ý nghĩa:</h5>
Accuracy đo lường tỷ lệ phần trăm mẫu được dự đoán đúng trong tổng số các mẫu. Do đó Accuracy đo lường tỷ lệ dự đoán đúng trên tổng số dự đoán của mô hình. Ví dụ, trong hệ thống phát hiện spam email, nếu trong 100 email, mô hình dự đoán đúng 90 email (Bao gồm cả các email spam và không spam), accuracy sẽ là 90%.
<h5 style="font-weight: 900">2. Công thức:</h5>

Accuracy = $\dfrac{\text{Số dự đoán đúng}}{\text{Tổng số mẫu}}$

Hoặc:

Accuracy = $\dfrac{TP + TN}{TP + TN + FP + FN}$

Trong đó:
- **TP (True Positives):** Số trường hợp dự đoán đúng là dương tính
- **FP (False Positives):** Số trường hợp dự đoán đúng là âm tính
- **TN (True Negatives):** Số trường hợp dự đoán nhầm là dương tính (Type I Error)
- **FN (False Negatives):** Số trường hợp dự đoán nhầm là âm tính (Type II Error)

**Lưu ý:** Accuracy không phù hợp khi dữ liệu bị mất cân bằng. Chẳng hạn, nếu 95% email là không spam, một mô hình đơn giản luôn dự đoán "không spam" sẽ có accuracy là 95% nhưng thực tế thì nó hoàn toàn vô dụng trong việc phát hiện spam


<h3 style="font-weight: 900">II. Precision (Độ chính xác):</h3>
<h5 style="font-weight: 900">1.Ý nghĩa:</h5>
Precision đo lường tỷ lệ các dự đoán đúng là dương tính trong tổng số các dự đoán đúng. Do đó Precision tập trung vào chất lượng của các dự đoán dương tính. Do đó Precision tập trung vào chất lượng của các dự đoán dương tính. Ví dụ về spam thì precision sẽ cho biết trong số các email mô hình dự đoán là spam thì sẽ có bao nhiêu thực sự là spam. Nếu mô hình dự đoán 10 email là spam, trong đó 8 email thực sự là spam thì precision sẽ là 80%. Precision cao nghĩa là khi mô hình nói một email là spam thì chúng ta có thể khá tin tưởng vào dự đoán đó. Điều này hữu ích trong các ứng dụng cần độ chính xác cao như chẩn đoán y tế.
<h5 style="font-weight: 900">2. Công thức:</h5>

Precision = $\dfrac{TP}{TP + FP}$



<h3 style="font-weight: 900">III. Recall (Độ nhạy):</h3>
<h5 style="font-weight: 900">1.Ý nghĩa:</h5>
Recall đo lường tỷ lệ các dự đoán đúng là dương tính (TP) mà mô hình có thể dự đoán đúng trên tổng số các mẫu dương tính thật sự. Vú dụ với spam, nếu có 20 email spam thực sự, và mô hình phát hiện được 15 trong số đó thì recall sẽ là 75%. Recall cao cho thấy mô hình ít bỏ sót các trường hợp cần phát hiện. Do đó, recall có ý nghĩa đặc biệt quan trọng trong các ngành an ninh, y tế.
<h5 style="font-weight: 900">2. Công thức:</h5>

Precision = $\dfrac{TP}{TP + FN}$



<h3 style="font-weight: 900">IV. F-score:</h3>
<h5 style="font-weight: 900">1.Ý nghĩa:</h5>
F-score là sự kết hợp cả precision và recall thành một chỉ số duy nhất, giúp cân bằng hai yếu tố trên. F-score thấp nếu một trong hai độ đo precision hoặc recall thấp và cao khi cả hai đều cao.

Các biến thể của F-score
- F1-score: Cân bằng giữa precision và recall
- F2-score: Đề cao recall hơn precision
- F0.5-score: Đề cao precision hơn recall

Trade-off giữa precision và recall:
Trong thực tê, thường sẽ có sự đánh đổi giữa precision và recall. Ví dụ, trong hệ thống phát hiện spam thì nếu ta muốn chắc chắn không bỏ sót email spam nào (recall cao) thì có thể ta phải chấp nhận việc một số email bình thường bị đánh nhầm là spam (precision thấp).

- Tăng threshold phân loại -> precision tăng, recall giảm
- Giảm threshold phân loại -> precision giảm, recall tăng

Ví dụ về trade-off trong thực tế: Hệ thống chẩn đoán ung thư:
- Threshold cao: Ít cảnh báo sai (precision cao) nhưng có thể bỏ sót ca bênh (recall thấp)
- Threshold thấp: Phát hiện nhiều ca bênh (recall cao) nhưng nhiều cảnh báo sai (precision thấp)
<h5 style="font-weight: 900">2. Công thức:</h5>

Precision = $\dfrac{2\times{\text{precision}}\times{\text{recall}} }{\text{precision + recall}}$

